# <center> Проект ML-инженера
## <center> Подготовка рекомендательной системы

Импорт библиотек

Чтение данных

In [8]:
import numpy as np
import pandas as pd
import scipy.sparse as sparse
import nmslib
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k
import pickle

In [9]:
ratings = pd.read_csv('data/ratings.csv')
books = pd.read_csv('data/books.csv')
tags = pd.read_csv('data/tags_cleaned.csv')
book_tags = pd.read_csv('data/book_tags.csv')

In [10]:
display(ratings.head(1))
display(books.head(1))
display(tags.head(1))
display(book_tags.head(1))

,user_id,book_id,rating
0,1,258,5


,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...


,tag_id,tag_name
0,509,19th-century


,goodreads_book_id,tag_id,count
0,1,30574,167697


In [11]:
# Словарь, с помощью которого можно находить book_id книги по goodreads_book_id
mapper = dict(zip(books.goodreads_book_id,books.book_id))
# добавим id книги в датафрейм book_tags
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags['id'] = book_tags.goodreads_book_id.apply(lambda x: mapper[x])

In [12]:
book_tags.head()

,goodreads_book_id,tag_id,count,id
1,1,11305,37174,27
4,1,33114,12716,27
5,1,11743,9954,27
6,1,14017,7169,27
10,1,27199,3857,27


In [13]:
# для работы с моделями в LightFM создаём разрежённые матрицы
#  будем хранить данные в формате COO (Coordinate List)
ratings_coo = sparse.coo_matrix((ratings.rating,(ratings.user_id, ratings.book_id)))
feature_ratings  = sparse.coo_matrix(([1]*len(book_tags), (book_tags.id, book_tags.tag_id)))

### Посторение модели

Объявим вспомогательные константы для обучения модели:

In [14]:
# число потоков процессора
NUM_THREADS = 1 # для Windows
# число параметров вектора
NUM_COMPONENTS = 60
# число эпох обучения
NUM_EPOCHS = 10
# зерно датчика случайных чисел
RANDOM_STATE = 42

Используем библиотеку LightFM, чтобы сделать матричное разложение (ALS) рейтингов книг и получить два набора векторов

In [15]:
# Разбиваем на обучающую и тестовую выборки
train, test = random_train_test_split(ratings_coo, test_percentage=0.2, random_state=RANDOM_STATE)

# Создаём модель
model = LightFM(
    learning_rate=0.05, # темп (скорость) обучения
    loss='warp', # loss-функция
    no_components=NUM_COMPONENTS, # размерность вектора признаков
    random_state=RANDOM_STATE # генератор случайных чисел
)

# Обучаем модель
model = model.fit(
    train, # обучающая выборка
    epochs=NUM_EPOCHS, # количество эпох обучения
    num_threads=NUM_THREADS, # количество потоков процессора
    item_features=feature_ratings # признаки товаров (рейтинги книг)
)

Тестирование модели

In [16]:
precision_score = precision_at_k(
    model, # модель
    test, # тестовая выборка
    num_threads=NUM_THREADS, # количество потоков процессора
    k=10, # количество предложений
    item_features=feature_ratings # признаки товаров
).mean() # усредняем результаты

recall_score = recall_at_k(
    model, # модель
    test, # тестовая выборка
    num_threads=NUM_THREADS, # количество потоков процессора
    k=10, # количество предложений
    item_features=feature_ratings # признаки товаров
).mean() # усредняем результаты

print(recall_score, precision_score)

0.04018157791747151 0.086986646


In [17]:
# Сохраним модель
with open('model9.pkl', 'wb') as file:
    pickle.dump(model, file, protocol=pickle.HIGHEST_PROTOCOL)

# Построение рекомендаций

## Добавим эмбеддинги к модели и посмотрим, что получилось

In [18]:
with open('model9.pkl', 'rb') as file:
    model = pickle.load(file)

In [19]:
# Достаём эбмеддинги
item_biases, item_embeddings = model.get_item_representations(features=feature_ratings)

print(item_biases.shape, item_embeddings.shape)

(10001,) (10001, 60)


In [20]:
#Инициализируем граф для поиска
nms_idx = nmslib.init(method='hnsw', space='cosinesimil')

# Добавляем книги в граф
nms_idx.addDataPointBatch(item_embeddings)
nms_idx.createIndex(print_progress=True)

In [21]:
# Вспомогательная функция для поиска по графу
# метод knnQuery() позволяет найти K ближайших соседей вектора в индексе (в графе)
def nearest_books_nms(book_id, index, n=10):
    nn = index.knnQuery(item_embeddings[book_id], k=n)
    return nn

### АНАЛИЗ РЕКОМЕНДАЦИЙ ПОСТРОЕННОЙ МОДЕЛИ

Для примера найдем id книги 1984

In [22]:
# Отфильтруем только те, где в названии встречается подстрока "1984"
books[books['title'].apply(lambda x: x.lower().find('1984')) >= 0]

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
12,13,5470,5470,153313,995,451524934,9.780452e+12,"George Orwell, Erich Fromm, Celâl Üster",1949.0,Nineteen Eighty-Four,...,1956832,2053394,45518,41845,86425,324874,692021,908229,https://images.gr-assets.com/books/1348990566m...,https://images.gr-assets.com/books/1348990566s...
845,846,5472,5472,2966408,51,151010269,9.780151e+12,"George Orwell, Christopher Hitchens",1950.0,Animal Farm & 1984,...,116197,118761,1293,1212,3276,16511,40583,57179,https://images.gr-assets.com/books/1327959366m...,https://images.gr-assets.com/books/1327959366s...
9795,9796,201145,201145,2563528,25,64440508,9.780064e+12,"Else Holmelund Minarik, Maurice Sendak",1968.0,A Kiss for Little Bear,...,11063,11604,126,87,284,1898,3053,6282,https://s.gr-assets.com/assets/nophoto/book/11...,https://s.gr-assets.com/assets/nophoto/book/50...


Теперь найдем все похожие книги и посмотрим на них

In [23]:
# Вызываем функцию для поиска ближайших соседей
print(nearest_books_nms(846, nms_idx))

(array([846,  14,  55, 809,  13,  48, 289, 271, 375,  28], dtype=int32), array([0.        , 0.03624159, 0.041255  , 0.05645806, 0.06419951,
       0.07261354, 0.08408093, 0.08804965, 0.08925718, 0.08993977],
      dtype=float32))


In [24]:
# Выделяем идентификаторы рекомендованных книг
nbm = nearest_books_nms(846, nms_idx)[0]
nbm

array([846,  14,  55, 809,  13,  48, 289, 271, 375,  28], dtype=int32)

In [25]:
# Посмотрим на авторов и названия рекомендованных книг
books[books.book_id.isin(nbm)][['authors', 'title']]

,authors,title
12,"George Orwell, Erich Fromm, Celâl Üster",1984
13,George Orwell,Animal Farm
27,William Golding,Lord of the Flies
47,Ray Bradbury,Fahrenheit 451
54,Aldous Huxley,Brave New World
270,Daniel Keyes,Flowers for Algernon
288,Richard Adams,"Watership Down (Watership Down, #1)"
374,Jack London,The Call of the Wild
808,"Aldous Huxley, Christopher Hitchens",Brave New World / Brave New World Revisited
845,"George Orwell, Christopher Hitchens",Animal Farm / 1984


Сохраним эмбеддинги

In [26]:
with open('item_embeddings.pkl', 'wb') as file:
    pickle.dump(item_embeddings, file, protocol=pickle.HIGHEST_PROTOCOL)